In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

url = 'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2024-3-1'
driver.get(url)

# Locate the table element at the bottom of the page using your specified XPath
table_element = driver.find_element('xpath', '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]')

# Extract the text from the table
table_text = table_element.text
print("Table at the bottom of the page:\n", table_text)

driver.quit()


Table at the bottom of the page:
 Time
Temperature
Dew Point
Humidity
Wind
Wind Speed
Wind Gust
Pressure
Precip.
Condition
1:51 AM 33 °F 11 °F 40 % NNW 13 mph 0 mph 30.46 in 0.0 in Fair
2:51 AM 32 °F 11 °F 42 % NW 13 mph 0 mph 30.47 in 0.0 in Fair
3:51 AM 32 °F 10 °F 40 % NW 9 mph 0 mph 30.47 in 0.0 in Fair
4:51 AM 31 °F 11 °F 43 % WNW 13 mph 0 mph 30.51 in 0.0 in Fair
5:51 AM 31 °F 11 °F 43 % NW 10 mph 0 mph 30.54 in 0.0 in Fair
6:51 AM 31 °F 11 °F 43 % NW 6 mph 0 mph 30.57 in 0.0 in Fair
7:51 AM 31 °F 12 °F 46 % NW 6 mph 0 mph 30.60 in 0.0 in Fair
8:51 AM 33 °F 12 °F 42 % NW 5 mph 0 mph 30.61 in 0.0 in Fair
9:51 AM 35 °F 13 °F 40 % NW 8 mph 0 mph 30.61 in 0.0 in Fair
10:51 AM 38 °F 14 °F 38 % 0 mph 0 mph 30.62 in 0.0 in Fair
11:51 AM 40 °F 14 °F 35 % NW 7 mph 0 mph 30.62 in 0.0 in Fair
12:51 PM 42 °F 13 °F 31 % S 6 mph 0 mph 30.61 in 0.0 in Fair
1:51 PM 43 °F 16 °F 34 % SSE 10 mph 0 mph 30.57 in 0.0 in Fair
2:51 PM 43 °F 17 °F 35 % S 14 mph 21 mph 30.56 in 0.0 in Partly Cloudy
3:51 P

In [3]:
copy_text = table_text
print(copy_text)

Time
Temperature
Dew Point
Humidity
Wind
Wind Speed
Wind Gust
Pressure
Precip.
Condition
1:51 AM 33 °F 11 °F 40 % NNW 13 mph 0 mph 30.46 in 0.0 in Fair
2:51 AM 32 °F 11 °F 42 % NW 13 mph 0 mph 30.47 in 0.0 in Fair
3:51 AM 32 °F 10 °F 40 % NW 9 mph 0 mph 30.47 in 0.0 in Fair
4:51 AM 31 °F 11 °F 43 % WNW 13 mph 0 mph 30.51 in 0.0 in Fair
5:51 AM 31 °F 11 °F 43 % NW 10 mph 0 mph 30.54 in 0.0 in Fair
6:51 AM 31 °F 11 °F 43 % NW 6 mph 0 mph 30.57 in 0.0 in Fair
7:51 AM 31 °F 12 °F 46 % NW 6 mph 0 mph 30.60 in 0.0 in Fair
8:51 AM 33 °F 12 °F 42 % NW 5 mph 0 mph 30.61 in 0.0 in Fair
9:51 AM 35 °F 13 °F 40 % NW 8 mph 0 mph 30.61 in 0.0 in Fair
10:51 AM 38 °F 14 °F 38 % 0 mph 0 mph 30.62 in 0.0 in Fair
11:51 AM 40 °F 14 °F 35 % NW 7 mph 0 mph 30.62 in 0.0 in Fair
12:51 PM 42 °F 13 °F 31 % S 6 mph 0 mph 30.61 in 0.0 in Fair
1:51 PM 43 °F 16 °F 34 % SSE 10 mph 0 mph 30.57 in 0.0 in Fair
2:51 PM 43 °F 17 °F 35 % S 14 mph 21 mph 30.56 in 0.0 in Partly Cloudy
3:51 PM 42 °F 19 °F 40 % SSE 18 mph 0 mp

In [128]:
import csv
import pandas as pd
#print(table_text)




# Split the table text into rows
rows = [row.split() for row in copy_text.strip().split('\n')[10:]]
# Specify the CSV file path
csv_file_path = 'table_data.csv'

# Write data to CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    csv_writer.writerows(rows)

print(f"Data has been written to {csv_file_path}")


# Read the CSV file into a list of lines
with open(csv_file_path, 'r') as file:
    lines = file.readlines()

# Modify each line to exclude the last value if it is the 19th value
modified_lines = [','.join(line.split(',')[:-1]) + '\n' if len(line.split(',')) == 19 else line for line in lines]

# Save the modified content back to the CSV file
with open(csv_file_path, 'w') as modified_file:
    modified_file.writelines(modified_lines)



# Read the CSV file into a DataFrame and drop the last column
df = pd.read_csv(csv_file_path)

# Display the DataFrame
#print(df.head())

df = df.iloc[:, [0, 1, 2, 4, 6, 9]]
column_names = df.columns.tolist()

# Create a new DataFrame with the column names as a row
column_names_row = pd.DataFrame([column_names], columns=df.columns)

# Concatenate the new row to the top of the original DataFrame
df_with_column_names = pd.concat([column_names_row, df]).reset_index(drop=True)
df_with_column_names.columns = ['time', 'am/pm', 'temp', 'dew point', 'humidity', 'wind speed']

# Display the resulting DataFrame
df_with_column_names.head()









Data has been written to table_data.csv


,time,am/pm,temp,dew point,humidity,wind speed
0,12:51,AM,33,11,40,13
1,1:51,AM,32,11,42,13
2,2:51,AM,32,10,40,9
3,3:51,AM,31,11,43,13
4,4:51,AM,31,11,43,10


In [133]:

df_with_column_names['time'] = df_with_column_names['time'].str.extract('(\d+)')

# Convert 'time' to numeric type
df_with_column_names['time'] = pd.to_numeric(df_with_column_names['time'])

# Display the resulting DataFrame
print(df_with_column_names)

# Assuming df_with_column_names is your DataFrame with the 'speed' column
# df_with_column_names = pd.read_csv('your_file.csv')  # Replace with the path to your CSV file if needed

# Custom function to update 'speed' column
def update_speed(cell):
    if cell == 'mph':
        return 0
    else:
        return cell

# Apply the custom function to the 'speed' column
df_with_column_names['speed'] = df_with_column_names['speed'].apply(update_speed)

# Display the resulting DataFrame
print(df_with_column_names)


AttributeError: Can only use .str accessor with string values!

Regex pattern did not match. Adjusted format: 9:51 AM 38 °F 14 °F 38, % 0, 0 mph 0, 0 mph 30.62 in, 0.0 in, Fair
9:51 AM 38 °F 14 °F 38, % 0, 0 mph 0, 0 mph 30.62 in, 0.0 in, Fair


In [ ]:
# train model to predict temp line based on the input so far, so train it to predict next temp values at next time step based on the past rows
# basically train it to find next row

